In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
from collections import Counter
import pickle

import codecs
import glob
import logging
import multiprocessing
import pprint

import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:

def cleanMessage(corpus_raw):
    corpus_raw = re.sub(r'\n+','.', corpus_raw)
    corpus_raw = re.sub(r'\.+', ". ", corpus_raw)
    corpus_raw = re.sub(r'…', " … ", corpus_raw)
    corpus_raw = re.sub(r'’', "", corpus_raw)
    corpus_raw = corpus_raw.replace("'", "")
    corpus_raw = re.sub(r' +', " ", corpus_raw)
    corpus_raw = re.sub(" +"," ", corpus_raw)
    return corpus_raw

Found files:  ['files/001.txt', 'files/002.txt', 'files/003.txt', 'files/004.txt', 'files/005.txt', 'files/006.txt', 'files/007.txt', 'files/008.txt']


In [119]:
#Read files
conv_filename = sorted(glob.glob('files/*.txt'))
print("Found files: ", conv_filename)

sonResponseDictionary = dict()
for conv_filename in conv_filename:
    print("Reading '{0}'...".format(conv_filename))
    with codecs.open(conv_filename, "r", "utf-8") as conv_file:
        allLines = conv_file.readlines()
        while '\n' in allLines: allLines.remove('\n')
        #convert to everything to lowercase
        [x.lower() for x in allLines]
        
        # FOR SON
        personName = "son"
        
        myMessage, otherPersonsMessage, currentSpeaker = "","",""
        for index,lines in enumerate(allLines):
            justMessage = lines
            colon = justMessage.find(':')
            # Find messages of the current person
            if (justMessage[:colon] == personName):
                if not myMessage:
                    startMessageIndex = index - 1
                myMessage += justMessage[colon+1:]
                if index == 1:
                    prevMessage = allLines[0]
                    colon = prevMessage.find(':')
                    otherPersonsMessage = prevMessage[colon+1:]
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    sonResponseDictionary[otherPersonsMessage] = myMessage
                    
            elif myMessage:
                # Now go and see what message the other person sent by looking at previous messages
                for counter in range(startMessageIndex, -1, -1):
                    currentLine = allLines[counter]
                    justMessage = currentLine
                    colon = justMessage.find(':')
                    if not currentSpeaker:
                        # Other speaker
                        currentSpeaker = justMessage[:colon]
                    elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
                        # A different person started speaking, so now I know that the first person's message is done
                        otherPersonsMessage = cleanMessage(otherPersonsMessage)
                        myMessage = cleanMessage(myMessage)
                        sonResponseDictionary[otherPersonsMessage] = myMessage
                        break
                    otherPersonsMessage = justMessage[colon+1:] + otherPersonsMessage
                myMessage, otherPersonsMessage, currentSpeaker = "","",""
        if myMessage:
            for counter in range(startMessageIndex, -1, -1):
                    currentLine = allLines[counter]
                    justMessage = currentLine
                    colon = justMessage.find(':')
                    if not currentSpeaker:
                        # Other speaker
                        currentSpeaker = justMessage[:colon]
                    elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
                        # A different person started speaking, so now I know that the first person's message is done
                        otherPersonsMessage = cleanMessage(otherPersonsMessage)
                        myMessage = cleanMessage(myMessage)
                        sonResponseDictionary[otherPersonsMessage] = myMessage
                        break
                    otherPersonsMessage = justMessage[colon+1:] + otherPersonsMessage

#Read files
conv_filename = sorted(glob.glob('files/*.txt'))
# print("Found files: ", conv_filename)

fatherResponseDictionary = dict()
for conv_filename in conv_filename:
#     print("Reading '{0}'...".format(conv_filename))
    with codecs.open(conv_filename, "r", "utf-8") as conv_file:
        allLines = conv_file.readlines()
        while '\n' in allLines: allLines.remove('\n')
        #convert to everything to lowercase
        [x.lower() for x in allLines]
        
        # FOR FATHER
        personName = "father"
        
        myMessage, otherPersonsMessage, currentSpeaker = "","",""
        for index,lines in enumerate(allLines):
            justMessage = lines
            colon = justMessage.find(':')
            # Find messages of the current person
            if (justMessage[:colon] == personName):
                if not myMessage:
                    startMessageIndex = index - 1
                myMessage += justMessage[colon+1:]
                if index == 1:
                    prevMessage = allLines[0]
                    colon = prevMessage.find(':')
                    otherPersonsMessage = prevMessage[colon+1:]
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    fatherResponseDictionary[otherPersonsMessage] = myMessage
                    
            elif myMessage:
                # Now go and see what message the other person sent by looking at previous messages
                for counter in range(startMessageIndex, -1, -1):
                    currentLine = allLines[counter]
                    justMessage = currentLine
                    colon = justMessage.find(':')
                    if not currentSpeaker:
                        # Other speaker
                        currentSpeaker = justMessage[:colon]
                    elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
                        # A different person started speaking, so now I know that the first person's message is done
                        otherPersonsMessage = cleanMessage(otherPersonsMessage)
                        myMessage = cleanMessage(myMessage)
                        fatherResponseDictionary[otherPersonsMessage] = myMessage
                        break
                    otherPersonsMessage = justMessage[colon+1:] + otherPersonsMessage
                myMessage, otherPersonsMessage, currentSpeaker = "","",""
        if myMessage:
            for counter in range(startMessageIndex, -1, -1):
                    currentLine = allLines[counter]
                    justMessage = currentLine
                    colon = justMessage.find(':')
                    if not currentSpeaker:
                        # Other speaker
                        currentSpeaker = justMessage[:colon]
                    elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
                        # A different person started speaking, so now I know that the first person's message is done
                        otherPersonsMessage = cleanMessage(otherPersonsMessage)
                        myMessage = cleanMessage(myMessage)
                        fatherResponseDictionary[otherPersonsMessage] = myMessage
                        break
                    otherPersonsMessage = justMessage[colon+1:] + otherPersonsMessage


Found files:  ['files/001.txt', 'files/002.txt', 'files/003.txt', 'files/004.txt', 'files/005.txt', 'files/006.txt', 'files/007.txt', 'files/008.txt']
Reading 'files/001.txt'...
Reading 'files/002.txt'...
Reading 'files/003.txt'...
Reading 'files/004.txt'...
Reading 'files/005.txt'...
Reading 'files/006.txt'...
Reading 'files/007.txt'...
Reading 'files/008.txt'...


In [121]:
len(sonResponseDictionary)
# len(fatherResponseDictionary)

294

In [ ]:
#turn words into tokens
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
#convert into a list of words
#remove unnnecessary,, split into words, no hyphens
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [ ]:
def processDataset(allLines):
    myStr = ""
    for line in allLines:
        myStr += line
    myStr = re.sub("[^a-zA-Z]"," ", myStr)
    finalDict = Counter(myStr.split())
    return myStr, finalDict

fullCorpus, datasetDictionary = processDataset(corpus_raw)
print('Finished parsing and cleaning dataset')
wordList = list(datasetDictionary.keys())

with open("data/wordList.txt", "wb") as fp:
	pickle.dump(wordList, fp)
print(len(wordList), 'unique words found')
print('Created file data/wordList.txt')

In [ ]:
len(wordList)

In [ ]:
raw_sentences = tokenizer.tokenize(corpus_raw)
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

# print(raw_sentences[890])
# print(sentence_to_wordlist(raw_sentences[890]))
token_count = sum([len(sentence) for sentence in sentences])
print("The chat corpus contains {0:,} tokens".format(token_count))

In [ ]:
raw_sentences = tokenizer.tokenize(corpus_raw)

In [ ]:
# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 0

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

chat2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

chat2vec.build_vocab(sentences)

print("Word2Vec vocabulary length:", len(chat2vec.wv.vocab))

In [ ]:
chat2vec.train(sentences, total_examples=chat2vec.corpus_count, epochs=10)

In [ ]:
np.save('data/embeddingMatrix.npy', chat2vec.wv.vectors)